In [9]:
import numpy as np
import torch
from smplpytorch.pytorch.smpl_layer import SMPL_Layer
from display_utils import display_model
import tqdm as notebook_tqdm
import torch
import sys
from manopth.manolayer import ManoLayer
from manopth import demo

In [12]:
data_type = "smplx"  
cuda = False
batch_size = 1
pca = False
body_shape = torch.rand(batch_size, 10)
if pca == True:
    ncomps = 6
elif pca == False:
    ncomps = 45

In [14]:
file = np.load("C:/Users/lalas/Downloads/val/val/npz/11JT4jRNI-o_12-2-rgb_front.npz")
smpl_layer = SMPL_Layer(center_idx=0, gender='neutral', model_root='smplpytorch/native/models')
mano_layer = ManoLayer(mano_root='mano/models', use_pca=pca, ncomps=ncomps, flat_hand_mean=False)

In [31]:
for frame_index in range(file['smplx_root_pose'].shape[0]):
    global_orient = file['smplx_root_pose'][frame_index]
    body_pose = file['smplx_body_pose'][frame_index]
    if data_type == "smplx":
        pose_params_h = torch.cat([torch.tensor(global_orient).view(1, -1), torch.tensor(body_pose).view(1, -1)], dim=1)
        hands = torch.zeros(1, 6)
        pose_params = torch.cat((pose_params_h, hands), dim=1)
    elif data_type == "smpl":
        pose_params = torch.cat([torch.tensor(global_orient).view(1, -1), torch.tensor(body_pose).view(1, -1)], dim=1)
    shape_params = torch.tensor(body_shape.reshape(batch_size, -1), dtype=torch.float32)
    verts, Jtr = smpl_layer(pose_params, th_betas=shape_params)
    #torch.save(Jtr, f'joint/body_{frame_index}.pt')
    #display_model({'verts': verts.cpu().detach(), 'joints': Jtr.cpu().detach()}, model_faces=smpl_layer.th_faces,with_joints=True, kintree_table=smpl_layer.kintree_table, savepath=f'pic/image_{frame_index}.png', show=False)

C:\Users\lalas\anaconda3\envs\smplpytorch\lib\site-packages\ipykernel_launcher.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if sys.path[0] == "":


In [29]:
for frame_index in range(file['smplx_root_pose'].shape[0]):
    lhand_pose_i = file['smplx_lhand_pose'][frame_index]
    rhand_pose_i = file['smplx_rhand_pose'][frame_index]
    root = torch.zeros(1,3)
    lhand_pose = torch.cat((root, torch.tensor(lhand_pose_i.reshape(1, -1))), dim=1)
    rhand_pose = torch.cat((root, torch.tensor(rhand_pose_i.reshape(1, -1))), dim=1)
    random_pose = torch.rand(batch_size, ncomps + 3)
    #print(random_pose.shape, lhand_pose.shape)
    lhand_verts, lhand_joints = mano_layer(lhand_pose, body_shape)
    rhand_verts, rhand_joints = mano_layer(rhand_pose, body_shape)
    #torch.save(lhand_joints, f'joint/lhand_{frame_index}.pt')
    #torch.save(rhand_joints, f'joint/rhand_{frame_index}.pt')
    #demo.display_hand({'verts': rhand_verts,'joints': rhand_joints},mano_faces=mano_layer.th_faces)
    #demo.display_hand({'verts': lhand_verts,'joints': lhand_joints},mano_faces=mano_layer.th_faces)